In [1]:
import sys
import os
sys.path.append('./src')
os.chdir(os.path.dirname(sys.path[0]))



# data Analysis 
import geopandas as gpd
import pandas as pd
import numpy as np



# plot
import matplotlib.pyplot as plt
import gc

# output
from output_image import write_output, analyze_model_result, output_to_csv


# multiprocessing
import multiprocessing
import brun


# model
from school_model import School

# spreadsheets
# import spreadsheets

#config
import configparser
import warnings
warnings.filterwarnings("ignore")

In [2]:
config_file_path_prefix = './config/'


school_params_ini = 'schoolparams.ini'
parser_school = configparser.ConfigParser()
parser_school.read(config_file_path_prefix + school_params_ini)
population_config = parser_school['SCHOOL_POPULATION']

# Parameter Setup
Here using a test trial small schedule with 5 steps of class per day
<br>
Run for 15 days

In [3]:

#change this to map path on comet later
map_path = "./layouts/schoollayout1/schoollayout1_processed.shp"
schedule_path = "./schedule_data/day_schedule.csv" #small schedule has only 5 steps per day for testing purposes
#full day schedule should be "day_schedule.csv" 
schedule_steps = 90 # full day_schedule steps should be 90


# if you wish to use google sheet parameter input, you'll have to follow instructions for getting credentials of the sheet api
# follow the first two steps in:
# https://developers.google.com/sheets/api/quickstart/python
# and save your credentials in the src folder as listed below
SHEET_URL = 'https://docs.google.com/spreadsheets/d/1Quyyey5B_kdQK1_OU0OkIDGZE27tuUfjZ6hCsV99-sM'
credentials = './src/credentials.json'


# two types of parameter setups available for batchrunner
# pre-setup for fixed/variable parameter dictionaries (consistant with mesa batchrunner)
######################
grade_N = eval(population_config['grade_N'])
KG_N =  eval(population_config['KG_N'])
preschool_N =  eval(population_config['preschool_N'])
special_education_N =  eval(population_config['special_education_N'])
faculty_N =  eval(population_config['faculty_N'])
seat_dist = 5
mask_prob =  eval(population_config['mask_prob'])
days = 15
max_steps = days*schedule_steps
iterations = 1
####################

# Single Run

In [5]:
%%time
school = School(map_path, schedule_path, grade_N, KG_N, preschool_N, special_education_N, 
                 faculty_N, seat_dist, init_patient=3, attend_rate=1, mask_prob=0.5, inclass_lunch=False)


while school.running and school.schedule.steps < max_steps:
    school.step()

params = "{'test': 0}" 
agent_df = school.datacollector.get_agent_vars_dataframe()
model_df = school.datacollector.get_model_vars_dataframe()

Wall time: 29min


In [ ]:
output_dfs = (model_df, agent_df)
output_to_csv(params, output_dfs, username='geoact')

# Simulation Result

In [ ]:
analyze_model_result(model_df=model_df) 